In [ ]:
# Dependencias

!pip install yfinance
!pip install EMD-signal==1.0.0

In [2]:
# imports e definições

from PyEMD import CEEMDAN
from datetime import timedelta, datetime
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

In [3]:
# date_index = stocks[stock].index

def set_ceemdan(ticker, imf, data, date_index, connection):
  features_in_order = ['Open', 'High', 'Low', 'Volume', 'Adj Close']
  table_name = ticker + '_' + imf
  pd_dataset = pd.DataFrame(data, columns=features_in_order)
  pd_dataset.set_index(date_index, inplace=True)
  pd_dataset.to_sql(table_name, con=connection, if_exists='replace')
  print(f'{table_name} stored in the database!')
  
  return table_name

def get_ceemdan(ticker, imf, connection):
  table_name = ticker + '_' + imf
  query = 'SELECT * from ' + table_name

  result = connection.execute(query)
  df = pd.DataFrame(result.fetchall())
  df.columns = result.keys()
  df.set_index('Date', inplace=True)

  return df

In [4]:
# Giving access to my google drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
 # This creates a symbolic link so that now the path to the database in GoogleDrive is equal to /mydb

!ln -s /content/gdrive/Shareddrives/TCC/Implementação/Capitulo3_Treinamento/db /mydb

In [6]:
!ls /mydb

13_14_15.db  14_15_16.db  15_16_17.db  Close  db  empty.db


In [7]:
from sqlalchemy import create_engine
my_conn = create_engine("sqlite:////mydb/13_14_15.db")

In [8]:
# Database connection (with all IMFs)

# for now it's a webscrapping and CEEMDAN decomposition

features_in_order = ['Open', 'High', 'Low', 'Volume', 'Adj Close'] # target feature must be the last one here
target_feature = 'Adj Close'

stocks_list = ['ABEV3.SA', 'BBAS3.SA', 'BBDC3.SA', 'BBSE3.SA', 'BRAP4.SA', 'BRFS3.SA', 'BRKM5.SA', 'BRML3.SA', 'BRPR3.SA', 'BRSR6.SA', 'AMER3.SA', 'CCRO3.SA', 'CESP6.SA', 'CIEL3.SA', 'CMIG4.SA', 'CPFE3.SA', 'CPLE6.SA', 'CSAN3.SA', 'CSNA3.SA', 'CYRE3.SA', 'DXCO3.SA', 'ECOR3.SA', 'ELET3.SA', 'EMBR3.SA', 'ENBR3.SA', 'EQTL3.SA', 'YDUQ3.SA', 'EVEN3.SA', 'EZTC3.SA', 'FIBR3.SA', 'GFSA3.SA', 'GGBR4.SA', 'GOAU4.SA', 'GOLL4.SA', 'HGTX3.SA', 'HYPE3.SA', 'IGTA3.SA', 'ITSA4.SA', 'ITUB4.SA', 'JBSS3.SA', 'KLBN11.SA', 'COGN3.SA', 'LAME4.SA', 'LIGT3.SA', 'LREN3.SA', 'MDIA3.SA', 'MGLU3.SA', 'MILS3.SA', 'MMXM3.SA', 'MRFG3.SA', 'MRVE3.SA', 'MULT3.SA', 'ODPV3.SA', 'OIBR3.SA', 'PCAR3.SA', 'PDGR3.SA', 'PETR3.SA', 'POMO4.SA', 'PSSA3.SA', 'ENAT3.SA', 'QUAL3.SA', 'RADL3.SA', 'RAPT4.SA', 'RENT3.SA', 'RSID3.SA', 'SANB3.SA', 'SBSP3.SA', 'SULA11.SA', 'TAEE11.SA', 'TIMS3.SA', 'TOTS3.SA', 'UGPA3.SA', 'USIM5.SA', 'VALE3.SA', 'VIVT3.SA', 'VLID3.SA', 'VVAR11.SA', 'WEGE3.SA' ]

def get_stock_infos(stock, start_date, end_date):
  yf.pdr_override()
  data = pdr.get_data_yahoo(stock, start_date, end_date).dropna()[features_in_order]
  return data

start_datetime = datetime(year=2013, month=1, day=1)
end_datetime = datetime(year=2015, month=12, day=31)

stocks = {}
for stock in stocks_list:
  stocks[stock.replace('.SA', '')] = get_stock_infos(stock, start_datetime, end_datetime)

ceemdan = CEEMDAN()   # They add noise in the paper.. should you add it too?
decomposed_data = {}
decomposed_stock_features_series = {}
scalers = {}
for stock in stocks:
  # loop das acoes
  print(f'{stock} Decompondo...')
  stock_dataframe = stocks[stock]
  decomposed_stock_features_series[stock] = {}
  scalers[stock] = {}
  for column in stock_dataframe.columns:
    # loop das features
    scaler = MinMaxScaler()
    decomposed_stock_features_series[stock][column] = {}
    #series = stock_dataframe[column].values                                          # (247,) 1D array
    series = stock_dataframe[column].values.reshape(-1,1)                             # (247, 1) 2D array
    scaler.fit(series)                                                                # Compute the minimum and maximum to be used for later scaling.
    scalers[stock][column] = scaler                                                   # MinMaxScaler(copy=True, feature_range=(0, 1))
    stock_feature_time_series = np.frombuffer(scaler.transform(series))               # scaler.transform(series) is the 2D normalized series and frombuffer makes it 1D
    stock_feature_time_series_imfs = ceemdan(stock_feature_time_series, max_imf=10)   # (5, 247) 247 is the number of values and 5 the number of imfs (which varies depending on the series)
    for i, imf_series in enumerate(stock_feature_time_series_imfs):
      # loop das IMF
      if i < len(stock_feature_time_series_imfs):                                     # If (5, 247), len is 247
        decomposed_stock_features_series[stock][column][f'IMF{i+1}'] = imf_series
      else:
        decomposed_stock_features_series[stock][column][f'Rsd'] = imf_series  


target_feature_max_imf_level = {}
series = {}

for stock in decomposed_stock_features_series:
  series[stock] = {}
  for feature in decomposed_stock_features_series[stock]:
    imfs = pd.DataFrame.from_dict(decomposed_stock_features_series[stock][feature])
    for imf in imfs:                                        # imf = 'IMF1' ou 'IMF2'
      if imf not in series[stock]:
        series[stock][imf] = []                             # {'PETR4': {'IMF1': [], 'IMF2': [], 'IMF3': [], 'IMF4': [], 'IMF5': [], 'IMF6': [], 'IMF7': []}}
      _series = imfs[imf].values                            # (247,)
      _series = _series.reshape((len(_series), 1))          # (247, 1)
      series[stock][imf] += [_series]                       # {'PETR4': {'IMF1': [imf_values_2D_array], 'IMF2': [], ...}} one 2D_array for each feature
      if feature == target_feature:
        target_feature_max_imf_level[stock] = imf        

series_cut = {}
for stock in series:
  if stock not in series_cut:
    series_cut[stock] = {}
  for imf_level_string in series[stock]:
    imf_level_int = int(imf_level_string[3:])
    if imf_level_int > int(target_feature_max_imf_level[stock][3:]):
      continue
    else:
      series_cut[stock][imf_level_string] = series[stock][imf_level_string].copy()

series = series_cut
dataset = {}

for stock in series:
  dataset[stock] = {}
  for imf_level in series[stock]:
    dataset[stock][imf_level] = np.hstack( tuple(series[stock][imf_level]) )
    if dataset[stock][imf_level].shape[1] == 5:     # Some IMFs don't have 5 columns, for some reason
      set_ceemdan(stock, imf_level, dataset[stock][imf_level], stocks[stock].index, my_conn)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/usr/local/lib/python3.7/dist-packages/PyEMD/CEEMDAN.py:183: RuntimeWarning: invalid value encountered in true_divide
  S = S/scale_s
/usr/local/lib/python3.7/dist-packages/PyEMD/CEEMDAN.py:183: RuntimeWarning: invalid value encountered in true_divide
  S = S/scale_s
/usr/local/lib/python3.7/dist-packages/PyEMD/CEEMDAN.py:183: RuntimeWarning: invalid value encountered in true_divide
  S = S/scale_s
/usr/local/lib/python3.7/dist-packages/PyEMD/CEEMDAN.py:183: RuntimeWarning: invalid value encountered in true_divide
  S = S/scale_s


HYPE3 Decompondo...
IGTA3 Decompondo...
ITSA4 Decompondo...
ITUB4 Decompondo...
JBSS3 Decompondo...
KLBN11 Decompondo...
COGN3 Decompondo...
LAME4 Decompondo...
LIGT3 Decompondo...
LREN3 Decompondo...
MDIA3 Decompondo...
MGLU3 Decompondo...
MILS3 Decompondo...
MMXM3 Decompondo...
MRFG3 Decompondo...
MRVE3 Decompondo...
MULT3 Decompondo...
ODPV3 Decompondo...
OIBR3 Decompondo...
PCAR3 Decompondo...
PDGR3 Decompondo...
PETR3 Decompondo...
POMO4 Decompondo...
PSSA3 Decompondo...
ENAT3 Decompondo...
QUAL3 Decompondo...
RADL3 Decompondo...
RAPT4 Decompondo...
RENT3 Decompondo...
RSID3 Decompondo...
SANB3 Decompondo...
SBSP3 Decompondo...
SULA11 Decompondo...
TAEE11 Decompondo...
TIMS3 Decompondo...
TOTS3 Decompondo...
UGPA3 Decompondo...
USIM5 Decompondo...
VALE3 Decompondo...
VIVT3 Decompondo...
VLID3 Decompondo...
VVAR11 Decompondo...
WEGE3 Decompondo...
ABEV3_IMF1 stored in the database!
ABEV3_IMF2 stored in the database!
ABEV3_IMF3 stored in the database!
ABEV3_IMF4 stored in the databa